In [6]:
search_for = 117
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.007016897201538086
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 340691351
type: [1, 1, 1, 1, 117] 117
cases of this type: 187388721
10000 23.190021990860544
20000 23.79617174174468
30000 23.15967408624694
40000 23.26516370779387
50000 91.32265363312453
60000 102.42505744739368
70000 85.00634814046614
80000 94.92372558014476
90000 92.48989299386606
100000 101.11977690169317
110000 102.63505091136369
120000 105.34142056700148
130000 108.53560871923474
140000 110.70127874206835
150000 146.2562868265822
160000 139.72895871219632
170000 133.11223258482352
180000 123.2296254004484
190000 128.99979254076027
200000 117.04879747893236
210000 128.01751778210905
220000 142.3589208003873
230000 122.51281513652839
240000 130.41874586690062
250000 136.84108026970338
260000 156.28709616948987
270000 156.3455552634054
280000 193.162994414223
290000 209.1730500212746
300000 252.10239528207117
310000 246.5618

3060000 279.9823891824909
3070000 297.8806614416871
3080000 295.7713158136358
3090000 274.29956166171
3100000 297.46801101277254
3110000 251.80802934035202
3120000 302.2316091894713
3130000 244.6678598967142
3140000 335.02333087290486
3150000 269.0172078183589
3160000 77.87744078767889
3170000 73.72502202581983
3180000 87.95889721095014
3190000 82.37237450415412
3200000 76.62240673913699
3210000 65.37511368750464
3220000 60.04365051418282
3230000 59.527422172440204
3240000 58.9578471151461
3250000 60.23388788889527
3260000 61.999821890862755
3270000 74.4351754019598
3280000 239.3748427335539
3290000 300.90742234387744
3300000 226.02299977551064
3310000 244.74192553276512
3320000 243.127282125962
3330000 239.25711187556064
3340000 265.3615670218393
3350000 248.48325366035175
3360000 271.71754722957013
3370000 235.1106717717591
3380000 282.6398792530785
3390000 242.95573232260062
3400000 267.9811190636757
3410000 247.22963611325434
3420000 263.9854611982012
3430000 253.60467768711112
344

6150000 260.56587480218195
6160000 237.625105448987
6170000 259.92424067849214
6180000 279.1852687138413
6190000 257.3937045429297
6200000 256.7925600792221
6210000 252.0907141960961
6220000 252.34142232704934
6230000 226.72043953082957
6240000 279.71022427575144
6250000 228.84259649384413
6260000 261.29997040348155
6270000 229.67394274688084
6280000 261.22430949558014
6290000 228.13214111713236
6300000 239.04492860163396
6310000 250.90322115971898
6320000 256.2365487766262
6330000 183.41869213470906
6340000 72.05357212633389
6350000 74.74577163095121
6360000 71.96757842703663
6370000 74.30982100748217
6380000 72.02458491734598
6390000 93.708957620348
6400000 213.2764577825929
6410000 278.3944252871428
6420000 274.24791390813147
6430000 352.08981031856894
6440000 167.99085985858017
6450000 57.81468394183999
6460000 58.66741539071762
6470000 58.244682036104834
6480000 60.50636079400617
6490000 58.46414296563688
6500000 73.62115467616613
6510000 237.3357565371022
6520000 301.034485280329

9240000 256.2538984914268
9250000 272.55266190157624
9260000 257.9474651671961
9270000 273.03107696171065
9280000 233.0313474101179
9290000 255.78452449120633
9300000 227.9923564252356
9310000 251.39492052908855
9320000 238.05334731732952
9330000 240.04598517492082
9340000 252.63855656406227
9350000 228.68883727226896
9360000 248.41615380624881
9370000 229.00614427656535
9380000 239.58400587621873
9390000 238.8259700268373
9400000 229.3539748538355
9410000 260.2977412290479
9420000 220.16235896227906
9430000 277.7784198413071
9440000 223.833334979389
9450000 270.61812415510116
9460000 227.6437820356715
9470000 263.9906306649308
9480000 229.5773439160809
9490000 268.79108503605926
9500000 283.0163635821016
9510000 77.05908248784586
9520000 73.97455257975564
9530000 71.4051205455633
9540000 92.26375488273678
9550000 71.05513981903468
9560000 73.65767714406277
9570000 88.42863480126044
9580000 333.76057171997724
9590000 257.1567578449436
9600000 288.6719955785311
9610000 222.1199876726572

12250000 247.02155929859197
12260000 243.26666768689574
12270000 220.05095646955627
12280000 217.31600826404997
12290000 244.06630642074128
12300000 221.96276621748996
12310000 249.9868144903325
12320000 225.58719814776794
12330000 232.4295077875272
12340000 250.3098985156514
12350000 234.70272504992465
12360000 247.59413193687922
12370000 227.08888964930708
12380000 247.98954639598415
12390000 245.79124725911151
12400000 244.8770329124792
12410000 232.2340384775335
12420000 239.77191863635954
12430000 251.79444804581468
12440000 234.7495923626895
12450000 261.7737690411194
12460000 245.3609591152179
12470000 258.3260563634781
12480000 243.4952715281945
12490000 250.3350663425731
12500000 227.29956939419935
12510000 236.71446968470644
12520000 238.98045847186722
12530000 227.0182035233729
12540000 239.33234711745172
12550000 224.06779767319813
12560000 229.2817573460659
12570000 229.91395860285655
12580000 241.44156653904162
12590000 245.80702632127017
12600000 241.71897573018973
12610

...

328430000 0.5976523881740968
328440000 1.1539034900220368
328450000 1.3025359254135358
328460000 3.104364207979381
328470000 3.105989337891254
328480000 3.562713419819309
328490000 2.2731216250514388
328500000 2.4972817218682564
328510000 3.6353821362434493
328520000 5.6687663302639715
328530000 4.101443019794988
328540000 2.592599433533496
328550000 2.6626327521312234
328560000 3.9245744493232175
328570000 4.319317692161811
328580000 3.1189658843391537
328590000 1.0298312426245742
328600000 0.9543942032421894
328610000 0.634969064142704
328620000 1.7606212866885596
328630000 2.8564315583626696
328640000 3.1729023696236016
328650000 3.3700117099810973
328660000 2.702108095484581
328670000 3.695917827802638
328680000 3.6342315792861974
328690000 2.4892152858237493
328700000 0.4906934599539041
328710000 0.49196606332296133
328720000 0.4900979917971161
328730000 0.48922193267065284
328740000 0.48953391388678547
328750000 0.48898211455568996
328760000 0.48890339837884905
328770000 0.487009

331280000 0.7362001586873332
331290000 2.1329798289514517
331300000 1.7088026470311815
331310000 3.281505098549584
331320000 1.5035491237563094
331330000 1.1225961200948424
331340000 2.0506093136108516
331350000 1.977013612894767
331360000 2.3124734647764895
331370000 1.8551386864597204
331380000 1.428857572934515
331390000 0.8858204770300521
331400000 1.3038434410209656
331410000 1.8689512980702387
331420000 1.8674995837290749
331430000 2.785978014405549
331440000 1.8696364503014287
331450000 0.5327440234385199
331460000 1.0627533580051263
331470000 1.6939501958676508
331480000 1.9450993564644918
331490000 1.8385262845040762
331500000 1.1687874063267774
331510000 0.737860691434357
331520000 1.5813214698159097
331530000 2.6468277433280085
331540000 2.0764761610079074
331550000 1.696625325067878
331560000 1.457753253339635
331570000 0.8596117324280871
331580000 2.3440041828391154
331590000 2.1416288213067785
331600000 1.9919023462036054
331610000 1.6812425403515099
331620000 0.639622550

334090000 0.3124791171338227
334100000 0.30579671853518486
334110000 0.30471821163431806
334120000 0.3264255572999782
334130000 0.2861915536222458
334140000 0.3177682143412895
334150000 0.436215456867377
334160000 1.4340237757282654
334170000 0.7252526726017662
334180000 0.6540069434307747
334190000 0.6572865930579702
334200000 0.9884020714134903
334210000 1.0575686953994963
334220000 0.7291632319257856
334230000 0.6326521025714675
334240000 0.7739228018985325
334250000 0.6114511281442842
334260000 0.5968349608855911
334270000 0.28457714145212043
334280000 0.28431620946019887
334290000 0.2949935003429718
334300000 0.3013063746854928
334310000 0.2888587780624628
334320000 0.2858971977124016
334330000 0.43552493162685635
334340000 0.7057749449803631
334350000 0.5740651914487945
334360000 0.5695394387802547
334370000 0.6568568728027145
334380000 0.7433864283623564
334390000 0.7233799639066525
334400000 0.6590702920609117
334410000 0.5362173708407084
334420000 0.6762431088395582
334430000 

336710000 0.21384510061538217
336720000 0.4780031300907797
336730000 0.20913739921415514
336740000 0.3322717004594803
336750000 0.20828706742506556
336760000 0.46223058454034727
336770000 0.3580293347257575
best so far: 0
type: [1, 9, 1, 13, 1] 117
cases of this type: 19773
336780000 0.2007189406697154
336790000 0.13584483640880052
best so far: 0
type: [1, 9, 13, 1, 1] 117
cases of this type: 1521
best so far: 0
type: [1, 13, 1, 1, 9] 117
cases of this type: 85293
336800000 0.18304708505618572
336810000 0.375678234862215
336820000 0.3752893332949744
336830000 0.15572992014348505
336840000 0.14567490915515688
336850000 0.14744300955703524
336860000 0.14958549347764252
336870000 0.22079854998703136
336880000 0.24577605553568732
best so far: 0
type: [1, 13, 1, 3, 3] 117
cases of this type: 28431
336890000 0.17375643382666509
336900000 0.16881515405515168
best so far: 0
type: [1, 13, 1, 9, 1] 117
cases of this type: 9477
336910000 0.16707737614775367
best so far: 0
type: [1, 13, 3, 1, 3] 1

339210000 0.45470829206852775
339220000 0.5264938907084067
339230000 0.2577626685882807
339240000 0.3898004580008917
339250000 0.2612578642530243
best so far: 0
type: [3, 1, 1, 3, 13] 117
cases of this type: 771147
339260000 0.1421142243885398
339270000 0.1392674669322438
339280000 0.06748969979723295
339290000 0.06325751525904734
339300000 0.23791533171347778
339310000 0.2266805653513471
339320000 0.13487607868585982
339330000 0.11801180639757052
339340000 0.06007498677012656
339350000 0.05990697251284122
339360000 0.13791980595059528
339370000 0.13231514754043022
339380000 0.12685872457188369
339390000 0.11060647297965157
339400000 0.058967310262872115
339410000 0.06406571531548103
339420000 0.12030152458640603
339430000 0.11191906123365294
339440000 0.1192100164949298
339450000 0.10361809206078448
339460000 0.05459295092611843
339470000 0.07309110007294019
339480000 0.20581391006417407
339490000 0.19856384946218464
339500000 0.20804642935262121
339510000 0.1550495381791128
339520000